In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import numpy as np
import pandas as pd
from tqdm import tqdm
import time 
import csv
import re

In [189]:
PROXY_ADDRESS = "185.199.228.220"
PROXY_PORT =7300
PROXY_USERNAME = "unnfmkfd"
PROXY_PASSWORD = "9xuuzq53da2d"
# 'Accept-Language' : 'en-US,en;q=0.9',
proxy_credential = {"https": f"http://{PROXY_USERNAME}:{PROXY_PASSWORD}@{PROXY_ADDRESS}:{PROXY_PORT}"}

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

In [3]:
country_url={'England':'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1',
             'Spain' : 'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1',
             'Italy':   'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1',
             'Germany':'https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1',
             'France': 'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1'}

In [4]:
links=[]
years=np.arange(2015,2022) 
for country, url in country_url.items():
    for year in years:
        s_url=url +'/plus/?saison_id='+ str(year) 
        r = requests.get(s_url,headers=headers,proxies=proxy_credential) 
        soup = BeautifulSoup(r.content, 'html.parser')

  
        table=soup.find('table', attrs={'class':'items'})
        trs=table.find_all('tr')  
        
        for tr in trs[2:]:
            td=tr.find_all('td')[1].find('a', href=True)['href']
            link='https://www.transfermarkt.com'+ td
            print(link)
            if link not in links:
                links.append(link)

https://www.transfermarkt.com/fc-chelsea/startseite/verein/631/saison_id/2015
https://www.transfermarkt.com/manchester-city/startseite/verein/281/saison_id/2015
https://www.transfermarkt.com/fc-arsenal/startseite/verein/11/saison_id/2015
https://www.transfermarkt.com/manchester-united/startseite/verein/985/saison_id/2015
https://www.transfermarkt.com/fc-liverpool/startseite/verein/31/saison_id/2015
https://www.transfermarkt.com/tottenham-hotspur/startseite/verein/148/saison_id/2015
https://www.transfermarkt.com/fc-everton/startseite/verein/29/saison_id/2015
https://www.transfermarkt.com/fc-southampton/startseite/verein/180/saison_id/2015
https://www.transfermarkt.com/west-ham-united/startseite/verein/379/saison_id/2015
https://www.transfermarkt.com/newcastle-united/startseite/verein/762/saison_id/2015
https://www.transfermarkt.com/leicester-city/startseite/verein/1003/saison_id/2015
https://www.transfermarkt.com/stoke-city/startseite/verein/512/saison_id/2015
https://www.transfermarkt.

#Get All players links

In [5]:
print(len(links))

686


In [6]:
players_links=[]

In [12]:
for link in links:
    
    club_page=requests.get(link,headers=headers,proxies=proxy_credential) 
    tree = html.fromstring(club_page.content)
    soup=BeautifulSoup(club_page.content, 'html.parser') 
    even_rows=soup.find('table',attrs={'class':'items'}).find('tbody').find_all('tr',attrs={'class':'even'})
    odd_rows=soup.find('table',attrs={'class':'items'}).find('tbody').find_all('tr',attrs={'class':'odd'}) 
    for row in even_rows:

      link='https://www.transfermarkt.com'+ row.find('td',attrs={'class':'posrela'}).find('table').find('td',attrs={'class':'hauptlink'}).find('a')['href']
      players_links.append(link)

    for row in odd_rows:
      link='https://www.transfermarkt.com'+ row.find('td',attrs={'class':'posrela'}).find('table').find('td',attrs={'class':'hauptlink'}).find('a')['href']
      players_links.append(link)

#Get players unique links

In [14]:
print(len(players_links))

25901


In [16]:
unique_players_links= []
for link in players_links:
  if link not in unique_players_links:
    unique_players_links.append(link)

In [17]:
print(len(unique_players_links))

9365


In [58]:
players_links=pd.DataFrame(data=unique_players_links)

In [59]:
players_links

,0
0,https://www.transfermarkt.com/asmir-begovic/pr...
1,https://www.transfermarkt.com/marco-amelia/pro...
2,https://www.transfermarkt.com/gary-cahill/prof...
3,https://www.transfermarkt.com/papy-djilobodji/...
4,https://www.transfermarkt.com/michael-hector/p...
...,...
9360,https://www.transfermarkt.com/jim-allevinah/pr...
9361,https://www.transfermarkt.com/jason-berthomier...
9362,https://www.transfermarkt.com/aiman-maurer/pro...
9363,https://www.transfermarkt.com/aboubakar-sidibe...


In [61]:
players_links.to_csv('unique_players_links.csv',index=False)

#transfer history

In [18]:
# transforming '16/17' to '2016'
def year_transformer(raw_year):
    year = raw_year.split('/')[0]
    if len(year) == 2:
        if int(year) <=22:
            return '20' + year
        else:
            return '19' + year
    elif len(year) == 4:
        return year

In [208]:
player_ids=[]
left_id=[]
joined_id=[]
MVs=[]
Fees=[]
seasons=[]
dates=[]

#first 1000 players

In [209]:
for link in unique_players_links[9000:]:
    #player id
    player_id=link.split('/')[-1]
    print(link)

    r = requests.get(link,headers=headers,proxies=proxy_credential) 
    soup = BeautifulSoup(r.content, 'html.parser')
    table=soup.find('div',attrs={'data-viewport':'Transferhistorie'}) 

    rows=table.find_all('div',attrs={'class':'grid tm-player-transfer-history-grid'}) 

    for row in rows[:-1]: 
        season=row.find('div', attrs={'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__season'}).text.rstrip().lstrip()
        season=year_transformer(season)


        date=row.find('div',attrs={'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__date'}).text.rstrip().lstrip()

        left=row.find('div',attrs={'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__old-club'}).find('a')['href'].split('/')[4]
        joined=row.find('div',attrs={'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__new-club'}).find('a')['href'].split('/')[4]

        MV=row.find('div',attrs={'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__market-value'}).text.rstrip().lstrip()
        Fee=row.find('div',attrs={'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__fee'}).text.rstrip().lstrip()


        player_ids.append(player_id)
        seasons.append(season)
        MVs.append(MV)
        Fees.append(Fee)
        left_id.append(left)
        joined_id.append(joined) 
        dates.append(date)

https://www.transfermarkt.com/theo-pionnier-bertrand/profil/spieler/811431
https://www.transfermarkt.com/lorenzo-prso/profil/spieler/865557
https://www.transfermarkt.com/jordan-lotomba/profil/spieler/313094
https://www.transfermarkt.com/antoine-mendy/profil/spieler/891998
https://www.transfermarkt.com/deji-sotona/profil/spieler/586599
https://www.transfermarkt.com/dan-ndoye/profil/spieler/365108
https://www.transfermarkt.com/malik-sellouki/profil/spieler/811430
https://www.transfermarkt.com/teddy-boulhendi/profil/spieler/789419
https://www.transfermarkt.com/flavius-daniliuc/profil/spieler/322655
https://www.transfermarkt.com/soudeysse-kari/profil/spieler/569365
https://www.transfermarkt.com/paul-wade/profil/spieler/696851
https://www.transfermarkt.com/salim-ben-seghir/profil/spieler/708681
https://www.transfermarkt.com/boubacar-fall/profil/spieler/836132
https://www.transfermarkt.com/abdoulaye-bakayoko/profil/spieler/855615
https://www.transfermarkt.com/tyrone-tormin/profil/spieler/496

In [210]:
info={}
info['player_id']=player_ids
info['MV']=MVs
info['Fee']=Fees
info['left']=left_id
info['joined']=joined_id
info['Date']=dates
info['season']=seasons
transfers=pd.DataFrame(data=info)

In [211]:
transfers.isnull().sum()

player_id    0
MV           0
Fee          0
left         0
joined       0
Date         0
season       0
dtype: int64

In [212]:
transfers

,player_id,MV,Fee,left,joined,Date,season
0,811431,€250k,free transfer,8158,56553,"Jul 1, 2022",2022
1,865557,€150k,?,8158,30200,"Aug 30, 2021",2021
2,313094,€4.00m,€7.00m,452,417,"Aug 3, 2020",2020
3,313094,€700k,free transfer,527,452,"Jul 1, 2017",2017
4,891998,€150k,-,8158,417,"Jul 1, 2022",2022
...,...,...,...,...,...,...,...
1350,503518,-,free transfer,26600,16654,"Oct 5, 2020",2020
1351,503518,-,free transfer,30200,26600,"Jul 1, 2019",2019
1352,503518,€50k,free transfer,16654,30200,"Oct 27, 2017",2017
1353,969153,€100k,free transfer,16654,14171,"Jul 1, 2022",2022


In [213]:
players_links[players_links[0]=='https://www.transfermarkt.com/robson-bambu/profil/spieler/458715'].index

Int64Index([8999], dtype='int64')

In [214]:
transfers.to_csv('9000-9365_tranfers.csv',index=False)

In [343]:
df1=pd.read_csv('0-2000_tranfers.csv')
df2=pd.read_csv('2000-3066_tranfers.csv')
df3=pd.read_csv('3066-3328_tranfers.csv')
df4=pd.read_csv('3328-4000_tranfers.csv')
df5=pd.read_csv('4000-5000_tranfers.csv')
df6=pd.read_csv('5000-6000_tranfers.csv')
df7=pd.read_csv('6000-6753_tranfers.csv')
df8=pd.read_csv('6753-7000_tranfers.csv')
df9=pd.read_csv('7000-8000_tranfers.csv')
df10=pd.read_csv('8000-9000_tranfers.csv')
df11=pd.read_csv('9000-9365_tranfers.csv')
merged=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11],axis=0, ignore_index=True)

In [344]:
merged.isnull().sum()

player_id    0
MV           0
Fee          2
left         0
joined       0
Date         0
season       0
dtype: int64

In [345]:
merged.shape

(105351, 7)

In [347]:
merged.drop_duplicates(keep='first', inplace=True)

In [348]:
merged.shape

(78987, 7)

In [349]:
merged

,player_id,MV,Fee,left,joined,Date,season
0,33873,€1.20m,free transfer,989,29,"Jul 20, 2021",2021
1,33873,€2.00m,End of loan,5,989,"Aug 31, 2020",2019
2,33873,€5.00m,Loan fee:€125k,989,5,"Jan 13, 2020",2019
3,33873,€5.00m,End of loan,10625,989,"Dec 31, 2019",2019
4,33873,€7.00m,loan transfer,989,10625,"Sep 2, 2019",2019
...,...,...,...,...,...,...,...
105346,503518,-,free transfer,26600,16654,"Oct 5, 2020",2020
105347,503518,-,free transfer,30200,26600,"Jul 1, 2019",2019
105348,503518,€50k,free transfer,16654,30200,"Oct 27, 2017",2017
105349,969153,€100k,free transfer,16654,14171,"Jul 1, 2022",2022


In [350]:
def change_date(x):
  x=x[:3]+','+x[3:]
  return x

In [351]:
merged['Date']=merged['Date'].apply(lambda x: change_date(x)) 

In [352]:
merged['MV']=merged['MV'].str.replace('€','')

In [353]:
index=merged[merged['MV'].str.contains('â‚¬')].index

In [354]:
index

Int64Index([86242, 86243, 87041], dtype='int64')

In [355]:
merged['MV'].loc[index]=merged['MV'].loc[index].apply(lambda x: x[3:])

<ipython-input-355-9619fd7c1a25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['MV'].loc[index]=merged['MV'].loc[index].apply(lambda x: x[3:])


In [356]:
merged['MV']=merged['MV'].apply(lambda x: float(x[:-1]) * 1000000 if 'm' in x else float(x[:-2]) * 1000000000 if 'bn' in x else float(x[:-1]) * 1000 if x.endswith('k') else x)

In [357]:
merged.rename(columns={'Fee':'Transfer_Fee'},inplace=True)

In [358]:
merged['Transfer_Fee']=merged['Transfer_Fee'].str.replace('€','')

In [359]:
merged['Transfer_Fee']=merged['Transfer_Fee'].str.replace('?','-')

<ipython-input-359-b2acc11911f0>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged['Transfer_Fee']=merged['Transfer_Fee'].str.replace('?','-')


In [360]:
index=merged[merged['Transfer_Fee'].str.contains('Loan fee',na=False)].index

In [366]:
merged['Transfer_Fee']=merged['Transfer_Fee'].astype(str)

In [367]:
merged['Loan_Fee']=merged['Transfer_Fee'].apply(lambda x:x.split(':')[1] if 'Loan fee' in x else 0 if 'loan' in x else '-')

In [371]:
merged['Transfer_Fee'].loc[index]=merged['Transfer_Fee'].loc[index].apply(lambda x: '-')

<ipython-input-371-b2c9d9c30d6c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['Transfer_Fee'].loc[index]=merged['Transfer_Fee'].loc[index].apply(lambda x: '-')


In [376]:
merged['Transfer_Fee']=merged['Transfer_Fee'].astype(str)

In [378]:
merged['Transfer_Fee']=merged['Transfer_Fee'].apply(lambda x: float(x[:-1]) * 1000000 if 'm' in x else float(x[:-2]) * 1000000000 if 'bn' in x else float(x[:-1]) * 1000 if x.endswith('k') else x)

In [382]:
merged['Loan_Fee']=merged['Loan_Fee'].astype(str)

In [383]:
merged['Loan_Fee']=merged['Loan_Fee'].apply(lambda x: float(x[:-1]) * 1000000 if 'm' in x else float(x[:-2]) * 1000000000 if 'bn' in x else float(x[:-1]) * 1000 if x.endswith('k') else x)

In [385]:
merged['Date']=pd.to_datetime(merged['Date'], format='%b, %d, %Y')

In [392]:
index=merged[merged['Transfer_Fee']=='End of loan'].index
merged['Transfer_Fee'].loc[index]='-'

<ipython-input-392-3e2185cad079>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['Transfer_Fee'].loc[index]='-'


In [394]:
merged.shape

(78987, 8)

In [395]:
merged.isnull().sum()

player_id       0
MV              0
Transfer_Fee    0
left            0
joined          0
Date            0
season          0
Loan_Fee        0
dtype: int64

In [397]:
merged.to_csv('transfer_history.csv',index=False)